# Make

This notebook contains code to covert ipynb notes to a renderable website which is hosted on github pages. You can see the output of this project at [sunitdeshpande.github.io/notes](https://sunitdeshpande.github.io/notes/)

## Cleaner

cleans autogenerated directories

In [1]:
import os
import shutil

class Cleaner(object):

    __DELETE_LIST = [
        'dist',
        'index.html'
    ]

    @staticmethod
    def clean():
        [
            shutil.rmtree(path) if os.path.isdir(path) else os.remove(path)
            for path in Cleaner.__DELETE_LIST 
            if os.path.exists(path)
        ]

## Coverter

Convert is a class which converts ipynb to other formats. The follow of the convert is as follows.

* Create appropriate folder under **dist** directory, if not exists.
* Covert .ipynb files to other format


In [2]:
import os
import glob

import nbconvert
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor

from jinja2 import Template

class HTMLConverter(object):

    __PATH_NOTEBOOK_TEMPLATE = 'template/note.tpl'
    __PATH_NOTE_TEMPLATE = 'template/note.html'
    __PATH_INDEX_TEMPLATE = 'template/index.html'
    __PATH_NOTES_FOLDER = 'notes'
    __PATH_OUTPUT_FOLDER = 'dist'    

    __OUTPUT_FOLDER_PREFIX = 'html'
    __OUTPUT_INDEX_FILE_PATH = 'index.html'

    __METADATA_NOTEBOOK_KEY = 'note_info'

    __html_exporter = None
    __execute_preprocessor = None;


    @property
    def html_exporter(self):
        # Create singletion html exporter
        if self.__html_exporter is None:        
            self.__html_exporter = nbconvert.HTMLExporter()
            self.__html_exporter.template_file = self.__PATH_NOTEBOOK_TEMPLATE

        return self.__html_exporter

    @property
    def execute_preprocessor(self):
        # Singletone execute preprocessor
        if self.__execute_preprocessor is None:
            self.__execute_preprocessor = ExecutePreprocessor(timeout=600, kernel_name='python3')
        
        return self.__execute_preprocessor


    def convert(self):
        generated_notes = []

        for filepath in self.__get_all_ipynb_files():

            self.__execute_ipynb_file(filepath)

            outputdir_path = self.__create_output_dir(filepath)
            
            notebook, metadata = self.__read_ipynb_file(filepath)
            
            body, resources = self.__convert_ipynb_notebook(notebook)
            
            output_file = self.__write_output_file(outputdir_path, metadata, body, resources)

            generated_notes.append({
                'html_file': output_file,
                'title': metadata['title'],
                'image': metadata['image'],
                'description': metadata['description']
            })
        
        # Create index file
        self.__write_index_file(generated_notes)


    def __get_all_ipynb_files(self):
        return [
            filepath 
            for filepath in glob.iglob('{rootdir}/**/*.ipynb'.format(rootdir=self.__PATH_NOTES_FOLDER), recursive=True)
        ]

    def __execute_ipynb_file(self, filepath):
        # Read notebook
        notebook, metadata = self.__read_ipynb_file(filepath)

        # Execute the code
        self.execute_preprocessor.preprocess(notebook, {
            'metadata':{
                'path': '.'
            }
        })

        # Save in original location
        with open(filepath, 'w') as file:
            nbformat.write(notebook, file)

    def __create_output_dir(self, filepath):

        # Make the outputdir path
        output_directory = os.path.dirname(os.path.join(self.__PATH_OUTPUT_FOLDER, self.__OUTPUT_FOLDER_PREFIX, filepath))

        # Create output directory if it not exists
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        return output_directory

    def __read_ipynb_file(self, inputfile):
        # Get file data
        with open(inputfile, 'r') as file:
            data = file.read()

        # Initialize the notebook
        notebook = nbformat.reads(data, as_version=4)

        # Get metadata
        metadata = notebook['metadata'][self.__METADATA_NOTEBOOK_KEY]
        metadata['filename'] = os.path.basename(inputfile) 

        return notebook, metadata

    def __convert_ipynb_notebook(self, notebook):
        # Convert notebook
        body, resources = self.html_exporter.from_notebook_node(notebook)

        return body, resources
        
    def __write_output_file(self, outputdir, metadata, body, resource):
        
        # Get meta information
        title = metadata.get('title', 'Notes')

        context = {
            'html_body': body,
            'inline_css': resource['inlining']['css'],

            'title': title
        }

        # Get outputfile name
        filename = '{name}.html'.format(name='.'.join(metadata['filename'].split('.')[:-1]))
        output_filename = os.path.join(outputdir, filename)

        slug_title = metadata.get('slug', None)
        if slug_title is not None:
            output_filename = os.path.join(outputdir, '{name}.html'.format(name=slug_title))
        
        # Render notes
        self.__render_template(output_filename, self.__PATH_NOTE_TEMPLATE, context)

        return output_filename
    
    def __write_index_file(self, notes):
        context = {
            'notes': notes
        }

        # Render index.html
        self.__render_template(self.__OUTPUT_INDEX_FILE_PATH, self.__PATH_INDEX_TEMPLATE, context)
  
    def __render_template(self, output_path, template_path, context={}):
        # Read template
        with open(template_path, 'r') as file:
            template_data = file.read()
        
        # Render template
        template = Template(template_data)
        rendered_data = template.render(**context)

        # Write output file
        with open(output_path, 'w') as file:
            file.write(rendered_data)


# Run one converter for now
HTMLConverter().convert()

## Main

Main entry point of the process.

In [6]:
import time

class Main(object):

    @staticmethod
    def start():
        print('Cleaning directory...')
        Cleaner.clean()
        print('Creating HTML notes...')
        HTMLConverter().convert()
        print('Done!!!')

Main.start()

Cleaning directory...
Creating HTML notes...
Done!!!
